This is to demonstrate how to use SigOpt software for hyperparameter tuning Logistic Regression model:

* After importing necessary modules, first create a connection to SigOpt API, you will need an account that you can sign up here: <https://sigopt.com/edu> and get the client token under the tab **API Tokens**
* Do the normal steps of Bayesian Optimization up till the creation of the function model_tune() which takes in the hyperparameters and return the best validation AUC (sidenote: since everything will be recorded at SigOpt API and you can download it as csv file later, you can get rid of recording searched parameters on your own)
* Then **important**: create an experiment with the connection to SigOpt API specifying the experiment name, the hypreparameters searching space, the type of hyperparameters (int, float, categorical), and finally observation budget (which is the number of iterations). I have all types in here already, so you can just copy as needed
* You can print out the experiment id link so you can easily check out experiment results
* Then **important**: you will need an function to pass the parameters you created in the experiments to model_tune(). Assignments are the default results from experiments, so you need to call 'assignments' dictionary for different hyperparameters
* Then **important**: create iterations(observations), the API will automatically update suggestions based on best result
* Then fetch the best experiment results after all iterations are done, but you can also manually retrieve the result at the experiment id link above
* Additional infor that might be need: delete an experiment (free trial account has 10 experiments limit, so be sure to manage your unwanted ones.
* The result: SigOpt gives a best validation auc of 0.797 so far. [see here](https://sigopt.com/guest?guest_token=BOSCVSNZUBMTYGXUPZNOQPNZQXGTVKIZVSRMTNHZBCALPQJK)

In [1]:
from __future__ import print_function
from __future__ import division

import string
import re
import random

import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchviz import make_dot, make_dot_from_trace

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle


#from bayes_opt import BayesianOptimization
import pandas as pd

In [2]:
from sigopt import Connection
conn = Connection(client_token='QULCGRMSXQAMRSTUPJPLAHDCHJMEMLXPSPZSQMWNEYKHLYJF')

In [3]:
torch.cuda.set_device(0)

In [4]:
torch.backends.cudnn.enabled=False

In [5]:
import model_lr1 as model #this changed
import Loaddata_final as Loaddata
import TrainVaTe_lr1 as TVT

# check GPU availability
use_cuda = torch.cuda.is_available()
use_cuda

True

In [6]:
# Load data set and target values
set_x = pickle.load(open('Data/h143.visits', 'rb'), encoding='bytes')
set_y = pickle.load(open('Data/h143.labels', 'rb'),encoding='bytes')


#if args.which_model == 'LR':
model_x = []
for patient in set_x:
    model_x.append([each for visit in patient for each in visit])  
    
#else: 
#    model_x = set_x  #this is for the rest of the models
    
merged_set= [[set_y[i],model_x[i]] for i in range(len(set_y))] #list of list or list of lists of lists
print("\nLoading and preparing data...")    
train1, valid1, test1 = Loaddata.load_data(merged_set)
print("\nSample data after split:")  
print(train1[1])
print("model is", 'LR1') #can change afterwards, currently on most basic RNN


Loading and preparing data...

Sample data after split:
[0, [491, 129, 813, 149, 84, 85, 491, 129, 813, 61, 84, 85, 358, 61, 84, 85, 491, 1250, 813, 61, 84, 85, 499, 61, 84, 85, 499, 61, 84, 85, 8004, 61, 84, 85, 2914, 61, 84, 85, 2914, 61, 84, 85, 2499, 499, 61, 84, 85, 499, 61, 84, 85, 1250, 129, 813, 61, 84, 85, 1250, 61, 84, 85]]
model is LR1


In [7]:
epochs =50 

In [8]:
#Hyperparamters to tune for LR: embed_dim, l2, lr. Define a function to return the validation AUC of the model 
def model_tune(embdim_exp, lr,l2, opt_code):
    #little transformations to use the searched values
    embed_dim = 2 ** int(embdim_exp) #base 2 
    lr = np.exp(lr) 
    l2 = np.exp(l2)
    ehr_model = model.EHR_LR(input_size =20000, embed_dim = embed_dim)  
    if use_cuda:
        ehr_model = ehr_model.cuda()
    
    if opt_code == 'Adadelta':
        opt= 'Adadelta'
        optimizer = optim.Adadelta(ehr_model.parameters(), lr=lr, weight_decay=l2) ## rho=0.9
    elif opt_code == 'Adagrad':
        opt= 'Adagrad'
        optimizer = optim.Adagrad(ehr_model.parameters(), lr=lr, weight_decay=l2) ##lr_decay no eps
    elif opt_code =='Adam':
        opt= 'Adam'
        optimizer = optim.Adam(ehr_model.parameters(), lr=lr, weight_decay=l2) ## Beta defaults (0.9, 0.999), amsgrad (false)
    elif opt_code =='Adamax':
        opt= 'Adamax'
        optimizer = optim.Adamax(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### Beta defaults (0.9, 0.999)
    elif opt_code =='RMSprop':
        opt= 'RMSprop'
        optimizer = optim.RMSprop(ehr_model.parameters(), lr=lr, weight_decay=l2 )                
    elif opt_code =='ASGD':
        opt= 'ASGD'
        optimizer = optim.ASGD(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### other parameters
    elif opt_code =='SGD':
        opt= 'SGD'
        optimizer = optim.SGD(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### other parameters
     
    
    #optimizer = optim.Adam(ehr_model.parameters(), lr=lr, weight_decay=l2)

    
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0

    for ep in range(epochs):
        current_loss, train_loss, _  = TVT.train(train1, model= ehr_model, optimizer = optimizer, batch_size = 1)
        avg_loss = np.mean(train_loss)
        valid_auc, y_real, y_hat, _  = TVT.calculate_auc(model = ehr_model, data = valid1, which_model = 'LR', batch_size = 1)
        if valid_auc > bestValidAuc: 
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            best_model= ehr_model
            #bestTestAuc, y_real, y_hat,_ = TVT.calculate_auc(model = ehr_model, data = test1, which_model = 'LR', batch_size = 1)

        if ep - bestValidEpoch > 12:
            break    
    bestTestAuc, y_real, y_hat,_ = TVT.calculate_auc(model = ehr_model, data = test1, which_model = 'LR', batch_size = 1)
    
    print(bestTestAuc)
    return bestValidAuc    

In [9]:
# create 3 different experiments for 3 different cells, keep the optimizer categorical 
experiment = conn.experiments().create(
  name='LR_AUC_Optimization',
  parameters=[
    dict(name='embdim_exp', type='int', bounds=dict(min=5, max=9)),
    #dict(name='hid_exp', type='int', bounds=dict(min=5, max=9)),
    #dict(name='layers_n', type='int', bounds=dict(min=1, max=3)),
    #dict(name='dropout', type='double', bounds=dict(min=0.1000, max= 0.9000)),
    dict(name='lr', type='double', bounds=dict(min=-11, max= -2)), 
    dict(name='l2', type='double', bounds=dict(min=-16, max= -1)),
    #dict(name='eps_exp', type='int', bounds=dict(min=-9, max=-4)),  
    dict(name='opt_code', type='categorical', categorical_values=[dict(name='Adadelta'), dict(name='Adagrad'),
                                                                  dict(name='Adam'), dict(name='Adamax'),dict(name='RMSprop'),
                                                                  dict(name='ASGD'), dict(name='SGD')]),
  ],
  observation_budget=100 #10~20 times number of parameters 
  )

In [10]:
print("Created experiment: https://sigopt.com/experiment/" + experiment.id);

Created experiment: https://sigopt.com/experiment/45007


In [11]:
# Evaluate your model with the suggested parameter assignments
def evaluate_model(assignments):
  return model_tune(assignments['embdim_exp'],# assignments['hid_exp'],assignments['layers_n'],assignments['dropout'],
                    assignments['lr'],assignments['l2'],assignments['opt_code'])

In [ ]:
#run the experiments, no need to modify
for i in range(100):
  suggestion = conn.experiments(experiment.id).suggestions().create()
  value = evaluate_model(suggestion.assignments)
  conn.experiments(experiment.id).observations().create(
    suggestion=suggestion.id,
    value=value,
  )

0.7289024297715341
0.5242180346499502
0.7679322344287479
0.5055706456338653
0.605624096963513
0.6009606690132131
0.7826039513463725
0.6389374240076016
0.7007067514019587
0.6064070345267578
0.6860350344843339
0.707919145583691
0.756779929277269
0.5042398013476099
0.6447505110578032
0.5781859970297867
0.5689439208392743
0.5811023646672433
0.5243373517238903
0.5531279157897487
0.7414195726776275
0.6513978658084721
0.6652848775024971
0.7870078731592198
0.8023906654479781
0.7062425197372876
0.7894163783793927
0.7877300983879754
0.5503083615501784
0.7054986066757188
0.7786493551982948
0.799838231882773
0.8067295917493229
0.7874501281825705
0.7693214794936442
0.716901851610624
0.7995258998954906
0.80113134341915
0.7566669349884436
0.7976162148300177
0.7878136543332018
0.7853957669157617
0.798390110130679
0.8012011660031595
0.7819969639753539
0.7913989454410272
0.801101973062488
0.8036554944186804
0.807061368000507
0.7928807887137334


In [ ]:
#Wrapping up the experiments and get the results
best_assignments_list = (
    conn.experiments(experiment.id)
        .best_assignments()
        .fetch()
)
if best_assignments_list.data:
    best_assignments = best_assignments_list.data[0].assignments

In [ ]:
#see final result 
best_assignment

In [ ]:
#delete an expriment 
experiment = conn.experiments('45006').delete()

In [ ]:
#tuned model
classifier = model_tune(assignments)